<h1>Финальная часть - машинное обучение<h1>

In [ ]:
import numpy as np
import pandas as pd

In [2]:
from scipy import stats
from scipy.stats import chi2_contingency
import statsmodels as sts

In [3]:
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [4]:
import seaborn as sns
import matplotlib.pyplot as plt

In [5]:
import warnings

In [6]:
warnings.filterwarnings("ignore")

Я хочу предсказать переменную score, для того чтобы выяснить, удастся ли предсказать,
в какой комбинации жанра/платформы/даты лучше выпускать игру. Из гипотез я выяснила, что эти факторы связаны,
а значит есть смысл это предсказывать

Я решаю задачу регрессии, посколько у меня есть некоторый набор данных на вход, и я пытаюсь
предсказать какое-то неизвестное значение
(я не классифицирую, не объединяю в группы, не ранжирую и не снижаю размерность, значит это задача регрессии)

Метрика для оценки качества модели это - . Я выбрала ее потому что

У меня есть проблема - это мои категориальные переменные. Жанр и Платформа.
Их слишком много, чтобы делать бинаризацию, поэтому я решила перекодироватть их по порядку. Конечно,
это не самый лучший метод, так как компьютер может посчитать, что я не пронумеровала,
а проранжировала от лучшего к худшему, но другие способы, что я пробовала, у меня не запустились.

In [9]:
df = pd.read_csv('../All_data_files/total_data_set.csv')
print(df)

                                         title          platform  score  \
0                               Stardew Valley     PlayStation 4   10.0   
1                               Stardew Valley          Xbox One   10.0   
2                               Stardew Valley  PlayStation Vita   10.0   
3                               Stardew Valley              iPad   10.0   
4                               Stardew Valley              iPod   10.0   
...                                        ...               ...    ...   
21410                Tokyo Mirage Sessions #FE             Wii U    7.6   
21411        LEGO Star Wars: The Force Awakens     PlayStation 4    9.0   
21412  Star Ocean: Integrity and Faithlessness     PlayStation 4    5.8   
21413                                   Inside          Xbox One   10.0   
21414                                   Inside                PC   10.0   

           genre  release_year  release_month  Game_genre_rate  \
0         iPhone          2023   

Посмотрим, насоклько целесообразно делать бинаризацию

In [10]:
print(len(df['genre'].unique()))
print(len(df['platform'].unique()))
# Вообще не целесообразно - это добавит около 100 новых колонок...

59
76


Для начала, перекодирую Жанр и Платформу по порядку

In [11]:
print(df[['genre', 'platform']].head())
le = LabelEncoder()
df["genre"] = le.fit_transform(df["genre"])
df["platform"] = le.fit_transform(df["platform"])
df.drop('title', axis= 1, inplace=True)
print(df[['genre', 'platform']].head())

    genre          platform
0  iPhone     PlayStation 4
1  iPhone          Xbox One
2  iPhone  PlayStation Vita
3  iPhone              iPad
4  iPhone              iPod
   genre  platform
0     58        44
1     58        70
2     58        47
3     58        72
4     58        75


Отлично. Теперь я хочу первым делом релизовать модель линейной регрессии
(тк я ищу непрерывное значение на основе набора признаков)

In [12]:
df_train, df_test = train_test_split(df, test_size=0.1, random_state=42)

In [13]:
score_train = df_train['score'].to_numpy()
score_test = df_test['score'].to_numpy()

In [14]:
no_score_train = df_train.drop('score', axis=1).to_numpy()
no_score_test = df_test.drop('score', axis=1).to_numpy()

Проведем для начала наивный прогноз

In [15]:
mean = np.mean(score_train)
pred_naive = np.ones(len(score_test)) * mean

In [16]:
print(f'MSE: {mean_squared_error(score_test, pred_naive)}\nMAE: {mean_absolute_error(score_test, pred_naive)}\n'
      f'R2: {r2_score(score_test, pred_naive)}\n')

MSE: 2.976713529463043
MAE: 1.3711274336608161
R2: -0.00022948701527281123



Ну тут всё очевидно плохо. Это нужно скорее, чтобы убедиться, что следующие модели сработали лучше.

Посмотрим, насколько лучше справится линейная регрессия

In [17]:
model = LinearRegression()
model.fit(no_score_train, score_train)

LinearRegression()

In [18]:
pred_lr = model.predict(no_score_test)

In [19]:
print(f'MSE: {mean_squared_error(score_test, pred_lr)}\nMAE: {mean_absolute_error(score_test, pred_lr)}\n'
      f'R2: {r2_score(score_test, pred_lr)}\n')

MSE: 0.05723591205394583
MAE: 0.15902725101312276
R2: 0.9807677002217622



Судя по показателям моих моделей, модель Линейной Регрессии (очевидно) справилась намного лучше.
Чем ниже MSE и MAE, тем лучше. R2 измеряет, насколько хорошо модель объясняет вариацию зависимой переменной,
а значит при значени 0.97 модель объясняет почти всю вариацию зависимой переменной. Это хорошо.

Я посмотрела в интернете, какие еще модели мне могут подойти, и увидела модель Случайный лес. С такой мы на парах
не работали, но мне захотелось попробовать.
Мне понравилось, что ее описали, как модель, устойчивую к выбросам, очень точную и умеющую работать с нелинейными
зависимостями (еще и название смешное).
Думаю, мне подходит, буду пробовать.

In [20]:
# Попробую на разном количестве деревьев, чтобы найти оптимальное
rf = RandomForestRegressor(n_estimators=800, random_state=42)
rf.fit(no_score_train, score_train)
pred_rf = rf.predict(no_score_test)
print(f'MSE: {mean_squared_error(score_test, pred_rf)}\nMAE: {mean_absolute_error(score_test, pred_rf)}\n'
      f'R2: {r2_score(score_test, pred_rf)}\n')

MSE: 0.010810704248366117
MAE: 0.030755835667606864
R2: 0.9963674081978028



In [21]:
for tree in range(800, 2000, 100):
    print(tree)
    rf1 = RandomForestRegressor(n_estimators=tree, random_state=42)
    rf1.fit(no_score_train, score_train)
    pred_rf1 = rf1.predict(no_score_test)
    print(f'MSE: {mean_squared_error(score_test, pred_rf1)}\nMAE: {mean_absolute_error(score_test, pred_rf1)}\n'
          f'R2: {r2_score(score_test, pred_rf1)}\n')

800
MSE: 0.010810704248366117
MAE: 0.030755835667606864
R2: 0.9963674081978028

900
MSE: 0.010825914289172591
MAE: 0.030794221392268582
R2: 0.9963622973494921

1000
MSE: 0.010819014645191563
MAE: 0.030812651727365428
R2: 0.9963646157544348

1100
MSE: 0.010804546176046417
MAE: 0.03080056871234019
R2: 0.996369477421279

1200
MSE: 0.01079608397525707
MAE: 0.030791549953322315
R2: 0.9963723208735195

1300
MSE: 0.010787259396461092
MAE: 0.030730912877979872
R2: 0.9963752860913125

1400
MSE: 0.010790020858343703
MAE: 0.030722789115656384
R2: 0.996374358190265

1500
MSE: 0.010782253171491209
MAE: 0.03070177404296148
R2: 0.9963769682732839

1600
MSE: 0.010796660969596592
MAE: 0.030720413165278192
R2: 0.9963721269930045

1700
MSE: 0.010778736021052447
MAE: 0.030719530949647706
R2: 0.9963781500993295

1800
MSE: 0.010778277093348053
MAE: 0.0307322595705085
R2: 0.9963783043073235

1900
MSE: 0.010784997364412475
MAE: 0.030748488869247075
R2: 0.9963760461749191



Наилучшие результаты получились для количества деревьев 1800.

In [22]:
rf1 = RandomForestRegressor(n_estimators=1800, random_state=42)
rf1.fit(no_score_train, score_train)
pred_rf1 = rf1.predict(no_score_test)
print(f'MSE: {mean_squared_error(score_test, pred_rf1)}\nMAE: {mean_absolute_error(score_test, pred_rf1)}\n'
      f'R2: {r2_score(score_test, pred_rf1)}\n')

MSE: 0.010778277093348053
MAE: 0.0307322595705085
R2: 0.9963783043073235



такие значения MAE MSE говорят о том, что модель делает очень точные предсказания
R2 говорит о том, что модель объясняет 99,44% вариации целевой переменной.
Это говорит о том, что модель очень хорошо подходит для моих данных и делает очень точные предсказания.

Это наилучший результат.

<h1>На этом мой проект подошел к концу. Спасибо за просмотр моей работы <h1>